In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('hi how are you ? ')

['hi', 'how', 'are', 'you', '?']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('hi')

'hi'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['fgwerg','test'],['test','gwerg','fgedgw'])

array([1., 0., 0.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['text'].tolist()

In [10]:
y = data['target'].tolist()

In [11]:
all_words = []
all_data = []
tags = []

In [12]:
for X_batch,y_batch in zip(X,y):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        X_batch_in_X_batch = stem(X_batch_in_X_batch)
        new_X.append(X_batch_in_X_batch)
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

In [13]:
all_words = sorted(set(all_words))

In [14]:
tags = sorted(set(tags))

In [15]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [16]:
X = []
y = []

In [17]:
for sentence,tag in all_data:
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    tag = tags.index(tag)
    y.append(tag)

In [18]:
from sklearn.model_selection import *

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [20]:
device = 'cuda'

In [21]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc